In [ ]:
# !pip install r5py

In [ ]:
# !pip install pyrosm
# !pip install folium
# !pip install geojson


In [32]:
import r5py
import os

# Ensure the file paths are correct
OSM_path = "/Users/max/Desktop/Transit_Dashboard/data/OSM_data/Toronto.osm.pbf" #"../../data/OSM_data/Toronto.osm.pbf"
GTFS_path = "/Users/max/Desktop/Transit_Dashboard/data/GTFS_data/raw/latest_feed_version_2024-10-22.zip"  #"../../data/GTFS_data/raw/latest_feed_version_2024-10-22.zip"


# Make the R5 transport network from these
%time transport_network = r5py.TransportNetwork(OSM_path, [GTFS_path])

# Uncomment the transport modes
transport_modes = [
    r5py.TransportMode.TRANSIT,
    r5py.TransportMode.WALK,
]

CPU times: user 2min 42s, sys: 6.66 s, total: 2min 49s
Wall time: 56.1 s


In [ ]:
import datetime

travel_time_matrix = r5py.TravelTimeMatrixComputer(
    transport_network,
    origins=origins,
    destinations=destinations,
    transport_modes=[r5py.TransportMode.TRANSIT],
    # departure=datetime.datetime(2019, 5, 13, 14, 0, 0),
).compute_travel_times()

In [15]:
# # pip install r5py.sampledata.sao_paulo
# import r5py
# import r5py.sampledata.sao_paulo

# transport_network = r5py.TransportNetwork(
#     r5py.sampledata.sao_paulo.osm_pbf,
#     [
#         r5py.sampledata.sao_paulo.gtfs,
#     ]
# )